In [15]:
import pandas as pd
import numpy as np

In [80]:
df_extract = pd.read_excel('/Users/inken/Desktop/result_mostlikeddataset_current.xlsx')
df_gemini = pd.read_csv('results/responses_subset_small_all_results_updated.csv')
df_moondream = pd.read_csv('SA_results_moondream_subset_5000.csv')
# df_moondream2 = pd.read_csv('SA_results_moondream_subset_5000.csv')
df_lens_t5 = pd.read_csv('results_lens/SA_results_lens_t5_subset_5000.csv')
df_vqa_base = pd.read_csv('results_VQA/SA_results_blip-vqa-base-large_subset_5000.csv')
df_vqa_capfilt = pd.read_csv('results_VQA/SA_results_blip-vqa-capfilt-large_subset_5000.csv')
df_text = pd.read_csv('results_text/results_text_roBERTa.csv')

In [64]:
df_extract10000 = pd.read_excel('/Users/inken/Desktop/result_randomdataset_1_dino.xlsx')
# df_gemini = pd.read_csv('results/responses_subset_small_all_results_updated.csv')
df_moondream1 = pd.read_csv('SA_results_moondream_random_dataset_pt1.csv')
df_moondream2 = pd.read_csv('SA_results_moondream_subset_pt2.csv')
# df_lens_t5 = pd.read_csv('results_lens/SA_results_lens_t5_subset_5000.csv')
# df_vqa_base = pd.read_csv('results_VQA/SA_results_blip-vqa-base-large_subset_5000.csv')
# df_vqa_capfilt = pd.read_csv('results_VQA/SA_results_blip-vqa-capfilt-large_subset_5000.csv')
df_text10000 = pd.read_csv('results_text/results_text_roBERTa_10000.csv')

In [7]:
dfs = [df_extract, df_gemini, df_moondream, df_lens_t5, df_vqa_base, df_vqa_capfilt, df_text]
df_names = ['df_extract', 'df_gemini', 'df_moondream', 'df_lens_t5', 'df_vqa_base', 'df_vqa_capfilt', 'df_text']

for i in range(len(dfs)):
    print(f"{df_names[i]}: {list(dfs[i].columns)}")
    print()

df_extract: ['image_name', 'object_detection_results', 'aisak_description', 'moondream_description', 'gemini_description', 'facial_emotion', 'text_similarity_semantic', 'text_similarity_semantic_expression', 'labels', 'bounding_boxes', 'bounding_box_confidence']

df_gemini: ['image', 'sentiment', 'confidence', 'explanation']

df_moondream: ['Unnamed: 0', 'image', 'sentiment']

df_lens_t5: ['Unnamed: 0', 'image', 'sentiment']

df_vqa_base: ['Unnamed: 0', 'image', 'sentiment']

df_vqa_capfilt: ['Unnamed: 0', 'image', 'sentiment']

df_text: ['created_at', 'img_name', 'language', 'like_count', 'quote_count', 'referenced_tweets', 'retweet_count', 'text', 'tweet_id', 'Negative', 'Neutral', 'Positive', 'text_sent']



In [81]:
# rename column sentiment --> model 5000

df_extract = df_extract.rename(columns={'image_name': 'image'})
df_gemini = df_gemini.rename(columns={'sentiment': 'gemini_img_sentiment'})
df_moondream = df_moondream.rename(columns={'sentiment': 'moondream_img_sentiment'})
df_lens_t5 = df_lens_t5.rename(columns={'sentiment': 'lens_t5_img_sentiment'})
df_vqa_base = df_vqa_base.rename(columns={'sentiment': 'vqa_base_img_sentiment'})
df_vqa_capfilt = df_vqa_capfilt.rename(columns={'sentiment': 'vqa_capfilt_img_sentiment'})
df_text = df_text.rename(columns={'text_sent': 'roberta_text_sentiment'})
df_text = df_text.rename(columns={'img_name': 'image'})

In [65]:
# 10000
df_extract10000 = df_extract10000.rename(columns={'image_name': 'image'})
df_moondream1 = df_moondream1.rename(columns={'sentiment': 'moondream_img_sentiment'})
df_moondream2 = df_moondream2.rename(columns={'sentiment': 'moondream_img_sentiment'})
df_text10000 = df_text10000.rename(columns={'text_sent': 'roberta_text_sentiment'})
df_text10000 = df_text10000.rename(columns={'img_name': 'image'})

In [82]:
print(df_gemini.duplicated().sum())
df_gemini = df_gemini.drop_duplicates()
print(df_gemini.duplicated().sum())


3000
0


In [66]:
def add_pg(text):
    full = str(text) + "pg"
    return full

df_text10000 = df_text10000.copy()
df_text10000['image'] = df_text10000['image'].apply(add_pg)

In [67]:
df_text10000.head()

,created_at,image,language,like_count,quote_count,referenced_tweets,retweet_count,text,tweet_id,Negative,Neutral,Positive,roberta_text_sentiment
0,2019-11-19T08:43:13.000Z,id_1196710509485187074_2019-11-19.jpg,en,7,0,NaN,2,Always keen to learn new useful abbreviations ...,1196710509485187074,0.074134,0.446448,0.479418,Positive
1,2019-06-03T16:31:05.000Z,id_1135584699009720320_2019-06-03.jpg,en,17,0,NaN,8,"b""Lots of people gathered around Barker's Pool...",1135584699009720320,0.842492,0.144292,0.013215,Negative
2,2019-02-15T13:27:10.000Z,id_1096400525833261063_2019-02-15.jpg,en,6,0,NaN,3,"b""#FollowFriday It's vital that you FOLLOW & R...",1096400525833261063,0.011891,0.194703,0.793405,Positive
3,2019-04-30T14:59:12.000Z,id_1123240390054879232_2019-04-30.jpg,en,3,0,NaN,1,Currently at Environmental Panel with speakers...,1123240390054879232,0.001468,0.143054,0.855477,Positive
4,2019-01-04T13:58:50.000Z,id_1081188206450757638_2019-01-04.jpg,en,20,0,NaN,17,"b""Avoid The 4 D's Of #ClimateChange Denial @An...",1081188206450757638,0.175964,0.752228,0.071808,Neutral


In [83]:
merged_df = df_extract.copy()

merged1 = pd.merge(merged_df, df_gemini[['image', 'gemini_img_sentiment']], on='image', how='left')
merged1 = pd.merge(merged1, df_moondream[['image', 'moondream_img_sentiment']], on='image', how='left')
merged2 = pd.merge(merged1, df_lens_t5[['image', 'lens_t5_img_sentiment']], on='image', how='left')
merged3 = pd.merge(merged2, df_vqa_base[['image', 'vqa_base_img_sentiment']], on='image', how='left')
merged4 = pd.merge(merged3, df_vqa_capfilt[['image', 'vqa_capfilt_img_sentiment']], on='image', how='left')
merged5 = pd.merge(merged4, df_text[['image', 'text', 'roberta_text_sentiment', 'like_count', 'quote_count', 'retweet_count']], on='image', how='left')

merged5.head()

,image,object_detection_results,aisak_description,moondream_description,gemini_description,facial_emotion,text_similarity_semantic,text_similarity_semantic_expression,labels,bounding_boxes,...,gemini_img_sentiment,moondream_img_sentiment,lens_t5_img_sentiment,vqa_base_img_sentiment,vqa_capfilt_img_sentiment,text,roberta_text_sentiment,like_count,quote_count,retweet_count
0,id_1153283149360762880_2019-07-22.jpg,poster,a screenshot of a cell phone with a message ab...,The image shows a screenshot of a webpage from...,The image is a screenshot of a web page. The ...,---,0.873518,low,NaN,[ 0.50041 0.50023 0.98887 0.99045],...,negative,Neutral,negative,positive,positive,the UN released a 740 page report compiled ove...,Negative,82582,3918,50280
1,id_1163744643600637952_2019-08-20.jpg,snow,trees are in the background,The image depicts a serene winter landscape wi...,There are 15 trees on the right side of the i...,---,0.906822,medium,NaN,[ 0.50014 0.78904 1 0.42132],...,neutral,Neutral,positive,positive,positive,"The Amazon Rainforest, one of the wettest plac...",Negative,69820,2456,51781
2,id_1122574040936452097_2019-04-28.jpg,glacier,a close up of a hole in a wall with a black an...,"The image shows a hole in a white wall, with a...",This is an image of a hole in a wall. The hol...,---,0.903128,medium,NaN,[ 0.50136 0.47235 0.23478 0.31739],...,negative,Negative,negative,neutral,neutral,just learned about climate change https://t.co...,Neutral,69235,87,11051
3,id_1188805167958974465_2019-10-28.jpg,person,araffe in a red suit falling down a snowy ramp,"In the image, a person dressed in a vibrant re...","The image shows a man in a purple suit, green...",disgust,0.881499,medium,NaN,[ 0.55921 0.55454 0.47932 0.57711],...,negative,Negative,negative,negative,negative,Climate change caused this. https://t.co/JG2Ly...,Negative,65465,70,6124
4,id_1108042949449969666_2019-03-19.jpg,"tree,podium,plant",spongebob squarepants - the tree in the middle...,The image depicts a vibrant underwater scene f...,The image is a cartoon. It shows a tree insid...,---,0.916996,medium,NaN,[ 0.48393 0.45347 0.3109 0.373...,...,neutral,Positive,negative,positive,positive,#GreenNewDeal haters\xe2\x80\x99 plan to addre...,Negative,62852,976,9145


In [68]:
#10000
moondream10000 = pd.concat([df_moondream1, df_moondream2])
print(moondream10000.shape)
moondream10000.head()

(10000, 3)


,Unnamed: 0,image,moondream_img_sentiment
0,0,id_1085840690846695424_2019-01-17.jpg,Negative
1,1,id_1083384161220657153_2019-01-10.jpg,Neutral
2,2,id_1091291744111681536_2019-02-01.jpg,Neutral
3,3,id_1094127385304330241_2019-02-09.jpg,Neutral
4,4,id_1095670082448097280_2019-02-13.jpg,Positive


In [69]:
df_extract10000['image'] = df_extract10000['image'].astype(str)
moondream10000['image'] = moondream10000['image'].astype(str)
df_text['image'] = df_text['image'].astype(str)

df_extract10000['image'] = df_extract10000['image'].str.strip()
moondream10000['image'] = moondream10000['image'].str.strip()
df_text['image'] = df_text['image'].str.strip()


In [74]:
# 10000
merged_df = df_extract10000.copy()

merged1 = pd.merge(merged_df, moondream10000[['image', 'moondream_img_sentiment']], on='image', how='left')
final = pd.merge(merged1, df_text10000[['image', 'text', 'roberta_text_sentiment']], on='image', how='left')
final.head()

,image,object_detection_results,aisak_description,moondream_description,gemini_description,facial_emotion,text_similarity_semantic,text_similarity_semantic_expression,labels,bounding_boxes,bounding_box_confidence,moondream_img_sentiment,text,roberta_text_sentiment
0,id_1196710509485187074_2019-11-19.jpg,"poster,person",there is a man standing at a podium giving a s...,A man in a blue shirt and black pants stands c...,This is a photo of a man giving a presentatio...,sadness,0.908977,medium,NaN,[ 0.22122 0.46297 0.43439 0.925...,"0.56562537, 0.5473341",Positive,Always keen to learn new useful abbreviations ...,Positive
1,id_1135584699009720320_2019-06-03.jpg,"tree,tree",people are gathered on the steps of a building...,The image depicts a large crowd of people gath...,This is a photo of a protest. There are about...,happy,0.919698,medium,crowded,[ 0.48808 0.47013 0.12871 0.201...,"0.47951597, 0.37119472",Negative,"b""Lots of people gathered around Barker's Pool...",Negative
2,id_1096400525833261063_2019-02-15.jpg,person,a close up of a statue of a woman with a veil on,The image depicts a statue of a woman draped i...,The image is a photo of a marble statue of a ...,disgust,0.916906,medium,NaN,[ 0.50008 0.57497 1 0.85058],0.4169812,Neutral,"b""#FollowFriday It's vital that you FOLLOW & R...",Positive
3,id_1123240390054879232_2019-04-30.jpg,NaN,people sitting in a classroom watching a prese...,The image depicts a classroom setting with a l...,This is a photo of a group of people sitting ...,sadness,0.885461,medium,crowded,NaN,NaN,Neutral,Currently at Environmental Panel with speakers...,Positive
4,id_1081188206450757638_2019-01-04.jpg,"poster,person",a cartoon of a man in a suit and tie with a sk...,The image is a black and white cartoon depicti...,The image is a cartoon. It is divided into fo...,---,0.900315,medium,NaN,[ 0.49571 0.49817 0.97641 0.983...,"0.5236316, 0.4189065",Negative,"b""Avoid The 4 D's Of #ClimateChange Denial @An...",Neutral


In [59]:
print(final['image'][2], df_text10000['image'][2])
print(final['image'][2] == df_text10000['image'][2])

id_1096400525833261063_2019-02-15.jpg id_1096400525833261063_2019-02-15.jpg
True


In [60]:
overlap = final[final['image'].isin(df_text10000['image'])]
overlap.shape

(10000, 13)

In [85]:
merged5.isna().sum()

image                                     0
object_detection_results                678
aisak_description                         0
moondream_description                     0
gemini_description                      636
facial_emotion                            0
text_similarity_semantic                  0
text_similarity_semantic_expression       0
labels                                 3775
bounding_boxes                          678
bounding_box_confidence                 678
gemini_img_sentiment                      0
moondream_img_sentiment                   0
lens_t5_img_sentiment                     0
vqa_base_img_sentiment                    0
vqa_capfilt_img_sentiment                 0
text                                      0
roberta_text_sentiment                   30
like_count                                0
quote_count                               0
retweet_count                             0
dtype: int64

In [84]:
# export dataframes 5000
merged5.to_excel('results/merged_results_5000.xlsx', index=False)

In [76]:
# export dataframes 10000
final.to_excel('results/merged_results_10000.xlsx', index=False)